# $(\mathrm{H}_2\mathrm{O})_5$ cost estimate

In [2]:
import numpy as np
import openfermion
import pyscf
from pyscf import gto

In [3]:
xyz_file = "TIP4P-5.xyz"

Get energy gap of a single molecule

In [4]:

atom = [('O', [-0.3511375701099324, -2.220327851906733, 1.4480207843187745]), ('H', [-0.24292674995680064, -1.697798925312798, 3.1763661875721043]), ('H', [-0.11012983703262759, -0.699680546250837, 0.4985624750191387]),]
basis = 'sto3g'
mol = gto.M(atom=atom,
            basis=basis,
            unit="Bohr")
rhf = mol.RHF.run()
cisolver = pyscf.fci.FCI(rhf)
cisolver.nroots = 5
e0, e1, e2, e3, e4 = cisolver.kernel()[0]
Delta = e1-e0
p = cisolver.ci[0][0,0]**2

converged SCF energy = -74.9629282604051


Generate Hamiltonian

In [5]:
from local_hamiltonian import get_localized_hamiltonian, get_localized_integrals
basis = 'sto3g'
method = 'lowdin'
mol = gto.M(atom="TIP4P-5.xyz",
            basis=basis)
rhf = mol.RHF.run()
e_rhf = rhf.e_tot
of_hamiltonian = get_localized_hamiltonian(mol, method)
constant, one_body_int, two_body_int = get_localized_integrals(mol, method)

converged SCF energy = -374.882033587153
converged SCF energy = -374.882033587154
converged SCF energy = -374.882033587153


Split the Hamiltonian

In [6]:
groups = [list(range(14*i, 14*i+14)) for i in range(5)]
original_qubit_hamiltonian = openfermion.transforms.jordan_wigner(of_hamiltonian)
constant_in_original_qubit_hamiltonian = original_qubit_hamiltonian.terms[()]
original_qubit_hamiltonian -= constant_in_original_qubit_hamiltonian
# Note that original_qubit_hamiltonian should not have constant hereafter. 

In [7]:
from utils import local_group_qubit_operator, get_coulomb_like_interaction_group
hamiltonian_list = []
for i in range(5):
    hamiltonian_list.append(local_group_qubit_operator(original_qubit_hamiltonian, groups[i]))
for i in range(5):
    hamiltonian_interaction = original_qubit_hamiltonian - hamiltonian_list[i]

Get the pauli coefficients

In [8]:
def coef2array(h):
    _array = np.array([h.terms[key] for key in h.terms])
    return _array[np.where(np.abs(_array)>1e-11)]
original_coef = coef2array(original_qubit_hamiltonian)
grouped_coef_list = []
for i in range(5):
    grouped_coef_list.append(coef2array(hamiltonian_list[i]))
interaction_coef = coef2array(hamiltonian_interaction)

Calculate norms

In [9]:
h1norm = abs(e_rhf-constant_in_original_qubit_hamiltonian)
for i in range(5):
    h1norm += np.sum(np.abs(grouped_coef_list[i]))
v1norm = np.sum(np.abs(interaction_coef))
v23norm = np.sum(np.abs(interaction_coef)**(2/3))**(3/2)
lvlh = len(interaction_coef)/(sum([len(grouped_coef_list[i]) for i in range(5)]))
print(h1norm)
print(v1norm)
print(v23norm)
print(lvlh)

569.5472345448393
437.855141426915
36797.35773543547
128.95971606033717


# Generate values in the paper

In [10]:
from cost_estimate import get_cost_for_first_order_pert, get_cost_for_second_order_pert
delta0 = 0.3e-3
delta1 = 0.3e-3
delta2 = 0.3e-3
r_scale = 1/5
epsilon_scale = 1/5
n_subsystems = 5
print(get_cost_for_first_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))
print(get_cost_for_second_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))

epsilon:  4.162502656980921e-08
n_filter:  (271645.97503196535+0j)
kappa:  0.0006991122173381573
M1:  559456.3504744264
r:  6.8515810736477276e-09
x_th: 5.26734187796994e-07
(30021299958540.66+0j)
r:  6.235396503312556e-12
epsilon_filter:  3.788155497742639e-11
epsilon_ptb:  8.912306111191753e-08
kappa:  0.0006991122173381573
w:  0.0006991122173381573
w0:  5.26734187796994e-07
M2:  344679095525483.2
n_filter:  (379296.8069607167+0j)
n_ptb:  (1774774.3937947417+0j)
(3.5714833262165623e+22+0j)
